In [1]:
import os
import json
import logging 

import openai
from dotenv import load_dotenv
from DuffelManager import get_best_offer

load_dotenv("../.env")
openai.api_key = os.getenv("OPENAI_API_KEY")
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

In [2]:
# write function to get flights for a day, location and airline
functions =  [
  {
    "name": "book_best_flight",
    "description": "Will return confirmation for a flight booked given a departure city, arrival city, departure date",
    "parameters": {
      "type": "object",
      "properties": {
        "departure_city": {
          "type": "string",
          "description": "The 3 letter IATA code for the airport in the city of departure e.g. JFK, LHR, CDG"
        },
        "destination_city": {
          "type": "string",
          "description": "The 3 letter IATA code for the airport in the city of arrival e.g. JFK, LHR, CDG"
        },
        "departure_date": {
          "type": "string",
          "description": "The date of departure in the format YYYY-MM-DD"
        },
        "time_of_day": {
          "type": "string",
          "enum": ["morning", "afternoon", "evening", "night"]
        },
        "airline_name": {
          "type": "string",
          "description": "The 2 letter IATA code for the airline the user would like to fly in e.g. DL, UA, AA"
        },
        "cabin_class": {
          "type": "string",
          "enum": ["first", "business", "premium_economy", "economy"]
        },
      },
      "required": ["departure_city", "destination_city", "departure_date"]
    }
  }
]

In [3]:
messages = [
  {"role": "system", "content": "You're a friendly flight booking assistant. You will be given a user query. Do your best to parse the user's request and respond accordingly. If the user hasn't given you enough information to make a function call, ask them for clarification"}
]

In [8]:
def user_interaction(query: str): 
  messages.append({"role": "user", "content": query})
  response = openai.chat.completions.create(
    model="gpt-4-1106-preview",
    messages=messages,
    functions=functions,
    function_call="auto"
  )
  reason = response.choices[0].finish_reason
  if reason == "function_call":
    # handle function call
    kwargs = json.loads(response.choices[0].message.function_call.arguments)
    
    logging.info(f"making function call with arguments: {kwargs}")
    
    response_obj = get_best_offer(**kwargs)
    if response_obj.success:
      best_offer = response_obj.best_offer
      # can be improved by feeding the response back to gpt and having it generate a response
      messages.append({"role": "assistant", "content": f"Great! I've booked you on {best_offer.airline_name} with {best_offer.num_segments - 1} stops from {kwargs['departure_city']} to {kwargs['destination_city']} on {best_offer.departure_time} in {best_offer.cabin_class} class for ${best_offer.total_amount}. Is there anything else I can help you with?"})
      return messages[-1]["content"]
    else:
      #run another gpt call here to format the error response better
      messages.append({"role": "assistant", "content": f"Sorry, I couldn't find any flights for {kwargs['departure_city']} to {kwargs['destination_city']} on {kwargs['departure_date']} in {kwargs['cabin_class']} class. Would you mind tweaking the parameters a bit?"})
      return messages[-1]["content"]
  else:
    # return this output to user
    messages.append({"role": "assistant", "content": response.choices[0].message.content})
    return messages[-1]["content"]

In [9]:
user_interaction("Hello, i'd like to book a flight from New York to Atlanta on the morning of 2023-12-25")

2023-11-10 12:48:32,917 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2023-11-10 12:48:32,919 - root - INFO - making function call with arguments: {'departure_city': 'JFK', 'destination_city': 'ATL', 'departure_date': '2023-12-25', 'time_of_day': 'morning'}
2023-11-10 12:48:32,919 - root - INFO - Getting offer for departure_city: JFK, destination_city: ATL, departure_date: 2023-12-25, time_of_day: morning, airline: None, cabin_class: None
2023-11-10 12:48:32,919 - root - INFO - Calling Duffel API
2023-11-10 12:48:51,517 - root - INFO - Getting best offer from 23 offers
2023-11-10 12:48:51,518 - root - INFO - Filtering for morning flights
2023-11-10 12:48:51,518 - root - INFO - Num offers after filtering time of day: 10
2023-11-10 12:48:51,519 - root - INFO - Returning best offer


"Great! I've booked you on JetBlue Airways with 0 stops from JFK to ATL on 2023-12-25 09:00:00 in economy class for 133.90. Is there anything else I can help you with?"

In [10]:
user_interaction("Actually, I'd like to fly in the evening")

2023-11-10 12:49:06,968 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2023-11-10 12:49:06,972 - root - INFO - making function call with arguments: {'departure_city': 'JFK', 'destination_city': 'ATL', 'departure_date': '2023-12-25', 'time_of_day': 'evening'}
2023-11-10 12:49:06,972 - root - INFO - Getting offer for departure_city: JFK, destination_city: ATL, departure_date: 2023-12-25, time_of_day: evening, airline: None, cabin_class: None
2023-11-10 12:49:06,972 - root - INFO - Calling Duffel API
2023-11-10 12:49:25,590 - root - INFO - Getting best offer from 23 offers
2023-11-10 12:49:25,591 - root - INFO - Filtering for evening flights
2023-11-10 12:49:25,592 - root - INFO - Num offers after filtering time of day: 2
2023-11-10 12:49:25,592 - root - INFO - Returning best offer


"Great! I've booked you on Duffel Airways with 0 stops from JFK to ATL on 2023-12-25 18:34:00 in economy class for 89.19. Is there anything else I can help you with?"